<a href="https://colab.research.google.com/github/YuriySamorodov/MIPT/blob/main/%D0%A1%D0%B5%D0%BC%D0%B5%D1%81%D1%82%D1%80%202/%D0%A6%D0%9A/%D0%9F%D1%80%D0%BE%D0%B4%D0%B2%D0%B8%D0%BD%D1%83%D1%82%D1%8B%D0%B5%20%D0%9C%D0%B5%D1%82%D0%BE%D0%B4%D1%8B%20%D0%9C%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE%20%D0%9E%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D1%8F/HW_6/%D0%A0%D0%B5%D1%88%D0%B0%D1%8E%D1%89%D0%B8%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D1%8C%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Считываем данные
df = pd.read_csv('light_music.csv')


In [2]:
from sklearn.model_selection import train_test_split

# Разделяем данные на признаки и целевую переменную
X = df.drop(columns=['popularity'])
y = df['popularity']

# Разбиваем данные на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [3]:
from sklearn.preprocessing import OrdinalEncoder

# Проверяем, есть ли категориальные строковые признаки
categorical_columns = X_train.select_dtypes(include=['object']).columns

if len(categorical_columns) > 0:
    encoder = OrdinalEncoder()
    X_train[categorical_columns] = encoder.fit_transform(X_train[categorical_columns])
    X_test[categorical_columns] = encoder.transform(X_test[categorical_columns])


In [4]:
train_shape = X_train.shape
test_shape = X_test.shape

print(f"Размерность тренировочной выборки: {train_shape}")
print(f"Размерность тестовой выборки: {test_shape}")


Размерность тренировочной выборки: (4000, 15)
Размерность тестовой выборки: (1000, 15)


In [5]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Создаем модель дерева решений
model = DecisionTreeRegressor(random_state=1)

# Обучаем модель на тренировочных данных
model.fit(X_train, y_train)

# Делаем предсказания на тестовых данных
y_pred = model.predict(X_test)

# Считаем MSE на тестовых данных
mse = mean_squared_error(y_test, y_pred)
print(f"MSE на тестовых данных: {mse}")


MSE на тестовых данных: 899.3786557955538


In [6]:
y_train_pred = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_train_pred)
print(f"MSE на тренировочных данных: {mse_train}")

MSE на тренировочных данных: 20.26219201619459


In [7]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

best_mse = float('inf')
best_depth = 0

for depth in range(1, 26):
    # Создаем модель дерева решений с текущей глубиной
    model = DecisionTreeRegressor(max_depth=depth, random_state=1)

    # Обучаем модель на тренировочных данных
    model.fit(X_train, y_train)

    # Делаем предсказания на тестовых данных
    y_pred = model.predict(X_test)

    # Считаем MSE на тестовых данных
    mse = mean_squared_error(y_test, y_pred)

    # Если MSE лучше, чем текущее лучшее значение, обновляем лучшую глубину и MSE
    if mse < best_mse:
        best_mse = mse
        best_depth = depth

print(f"Лучшая глубина дерева: {best_depth}")
print(f"Лучшее значение MSE на тесте: {round(best_mse, 2)}")

Лучшая глубина дерева: 10
Лучшее значение MSE на тесте: 705.91


In [8]:
# Преобразуем целевой признак в бинарный вид
y_train_bin = (y_train > 50).astype(int)
y_test_bin = (y_test > 50).astype(int)

In [9]:
# Соотношение классов на обучающей выборке
train_class_1 = sum(y_train_bin == 1)
train_class_0 = sum(y_train_bin == 0)

# Соотношение классов на тестовой выборке
test_class_1 = sum(y_test_bin == 1)
test_class_0 = sum(y_test_bin == 0)

print(f"Объектов 1 класса на обучении: {train_class_1}")
print(f"Объектов 0 класса на обучении: {train_class_0}")
print(f"Объектов 1 класса на тесте: {test_class_1}")
print(f"Объектов 0 класса на тесте: {test_class_0}")

Объектов 1 класса на обучении: 1627
Объектов 0 класса на обучении: 2373
Объектов 1 класса на тесте: 401
Объектов 0 класса на тесте: 599


In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

best_accuracy = 0
best_depth = 0

for depth in range(1, 26):
    # Создаем модель дерева решений с текущей глубиной
    model = DecisionTreeClassifier(max_depth=depth, random_state=1)

    # Обучаем модель на тренировочных данных
    model.fit(X_train, y_train_bin)

    # Делаем предсказания на тестовых данных
    y_pred = model.predict(X_test)

    # Считаем accuracy на тестовых данных
    accuracy = accuracy_score(y_test_bin, y_pred)

    # Если accuracy лучше, чем текущее лучшее значение, обновляем лучшую глубину и accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth

print(f"Лучшая глубина дерева: {best_depth}")
print(f"Лучшее значение accuracy на тесте: {round(best_accuracy, 5)}")

Лучшая глубина дерева: 21
Лучшее значение accuracy на тесте: 0.697


In [12]:
# Обучаем модель с лучшей глубиной
best_depth = 10  # Замените на значение, полученное в предыдущем задании
model = DecisionTreeClassifier(max_depth=best_depth, random_state=1)
model.fit(X_train, y_train_bin)

# Получаем важности признаков
feature_importances = model.feature_importances_

# Создаем DataFrame для удобства анализа
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})

# Сортируем по важности
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)

             Feature  Importance
14       track_genre    0.143549
8       acousticness    0.101842
5           loudness    0.095755
0        duration_ms    0.088352
11           valence    0.082134
9   instrumentalness    0.078435
3             energy    0.076570
7        speechiness    0.068388
10          liveness    0.067458
2       danceability    0.061187
12             tempo    0.058852
4                key    0.039635
1           explicit    0.018448
13    time_signature    0.014127
6               mode    0.005268
